In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('selected_data_v2.csv', index_col = 0)

In [3]:
df

,card_zip_total_7,card_state_total_3,card_zip_total_14,card_state_total_1,card_zip_total_30,card_zip_max_30,card_merch_max_14,card_state_max_14,merch_zip_total_1,card_zip_max_1,...,merch_zip_max_1,card_zip_max_0,Cardnum_max_7,Merchnum_total_7,Cardnum_total_14,Cardnum_max_14,card_merch_avg_3,merch_zip_max_7,card_zip_avg_14,Label
Recnum,,,,,,,,,,,,,,,,,,,,,
1,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,0
2,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,...,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,0
3,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,...,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,0
4,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,7.24,3.62,...,3.62,3.62,3.62,7.24,3.62,3.62,3.62,3.62,3.62,0
5,7.24,7.24,7.24,7.24,7.24,3.62,3.62,3.62,10.86,3.62,...,3.62,3.62,3.62,10.86,7.24,3.62,3.62,3.62,3.62,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96749,84.79,84.79,84.79,84.79,84.79,84.79,84.79,84.79,84.79,84.79,...,84.79,84.79,84.79,84.79,84.79,84.79,84.79,84.79,84.79,0
96750,118.75,886.25,718.60,118.75,718.60,599.85,599.85,1385.00,118.75,118.75,...,118.75,118.75,1478.00,118.75,8089.20,1705.60,118.75,118.75,359.30,0
96751,363.56,435.41,363.56,435.41,363.56,363.56,363.56,363.56,5780.89,363.56,...,2487.98,363.56,1065.73,6442.23,2578.13,1065.73,363.56,2487.98,363.56,0


In [4]:
trn_tst = df.loc[:84299, :].copy()
oot = df.loc[84300:,:].copy()

In [5]:
X = trn_tst.iloc[:,:-1]
y = trn_tst.iloc[:,-1]
X_oot = oot.iloc[:,:-1]
y_oot = oot.iloc[:,-1]

In [6]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X_oot = scaler.transform(X_oot)

In [7]:
## transfer back to dataframe since scaler returns arrays
## we need the functionality of dataframe in the function below

trn_tst = pd.DataFrame(X, columns = trn_tst.columns[:-1], index = trn_tst.index)
trn_tst['Label'] = y.values
oot = pd.DataFrame(X_oot, columns = oot.columns[:-1], index = oot.index)
oot['Label'] = y_oot.values

In [8]:
X = trn_tst.iloc[:,:-1]
y = trn_tst.iloc[:,-1]
X_oot = oot.iloc[:, :-1]

In [9]:
def FDR3(data, predictions):
    result = data.copy()
    result['prediction'] = predictions[:,1]
    result = result.sort_values('prediction', ascending=False)
    topRows = int(round(len(result)*0.03))
    return round(result.head(topRows).Label.sum()/result.Label.sum(),4)

In [10]:
nitermax = 10

## 1. Baseline Logistics

In [11]:
from sklearn.linear_model import LogisticRegression

In [12]:
fdr3_train = []
fdr3_test = []
fdr3_oot = []



for niter in range(nitermax):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    model = LogisticRegression(verbose = 0)
    
    model.fit(X_train, y_train)
    
    train, test = X_train.copy(), X_test.copy()
    train['Label'] = y_train.values
    test['Label'] = y_test.values
    
    ## predict on training set
    pred = model.predict_proba(X_train)
    fdr3_train.append(FDR3(train, pred))
    
    ## predict on test set
    pred = model.predict_proba(X_test)
    fdr3_test.append(FDR3(test, pred))
    
    ## predict on oot
    pred = model.predict_proba(X_oot)
    fdr3_oot.append(FDR3(oot, pred))

lr_df = pd.DataFrame(data=[[np.mean(fdr3_train), np.mean(fdr3_test), np.mean(fdr3_oot)]],\
                     columns=['FDR3 train', 'FDR3 test', 'FDR3 oot'])

/Users/aslanshi/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/aslanshi/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/aslanshi/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/aslanshi/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/aslanshi/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defau

In [13]:
lr_df

,FDR3 train,FDR3 test,FDR3 oot
0,0.68562,0.67943,0.3609


In [13]:
lr_df

,FDR3 train,FDR3 test,FDR3 oot
0,0.68894,0.67815,0.35418


## 2. MLP

In [14]:
from sklearn.neural_network import MLPClassifier

In [15]:
fdr3_train = []
fdr3_test = []
fdr3_oot = []

for niter in range(nitermax):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    model = MLPClassifier(hidden_layer_sizes=(5,),
                          activation='logistic',
                          max_iter=200,
                          alpha=0.0001,
                          solver='sgd',
                          tol=0.00001,
                          learning_rate='adaptive',
                          learning_rate_init=0.2)
    
    model.fit(X_train, y_train)
    
    train, test = X_train.copy(), X_test.copy()
    train['Label'] = y_train.values
    test['Label'] = y_test.values
    
    ## predict on training set
    pred = model.predict_proba(X_train)
    fdr3_train.append(FDR3(train, pred))
    
    ## predict on test set
    pred = model.predict_proba(X_test)
    fdr3_test.append(FDR3(test, pred))
    
    ## predict on oot
    pred = model.predict_proba(X_oot)
    fdr3_oot.append(FDR3(oot, pred))

mlp_df = pd.DataFrame(data=[[np.mean(fdr3_train), np.mean(fdr3_test), np.mean(fdr3_oot)]],\
                     columns=['FDR3 train', 'FDR3 test', 'FDR3 oot'])

/Users/aslanshi/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/aslanshi/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/aslanshi/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/aslanshi/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the op

In [16]:
mlp_df

,FDR3 train,FDR3 test,FDR3 oot
0,0.78724,0.77033,0.55476


In [17]:
mlp_df

,FDR3 train,FDR3 test,FDR3 oot
0,0.79374,0.76436,0.56089


## 3. Boosted Trees

In [18]:
from sklearn.ensemble import GradientBoostingClassifier

In [19]:
fdr3_train = []
fdr3_test = []
fdr3_oot = []

for niter in range(nitermax):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    model = GradientBoostingClassifier(n_estimators=300,
                                       max_depth = 4)
    
    model.fit(X_train, y_train)
    
    train, test = X_train.copy(), X_test.copy()
    train['Label'] = y_train.values
    test['Label'] = y_test.values
    
    ## predict on training set
    pred = model.predict_proba(X_train)
    fdr3_train.append(FDR3(train, pred))
    
    ## predict on test set
    pred = model.predict_proba(X_test)
    fdr3_test.append(FDR3(test, pred))
    
    ## predict on oot
    pred = model.predict_proba(X_oot)
    fdr3_oot.append(FDR3(oot, pred))

bt_df = pd.DataFrame(data=[[np.mean(fdr3_train), np.mean(fdr3_test), np.mean(fdr3_oot)]],\
                     columns=['FDR3 train', 'FDR3 test', 'FDR3 oot'])

In [21]:
bt_df

,FDR3 train,FDR3 test,FDR3 oot
0,0.99457,0.8809,0.44804


## 4. Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
fdr3_train = []
fdr3_test = []
fdr3_oot = []

for niter in range(nitermax):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    model = RandomForestClassifier(bootstrap=True,
                                   max_depth=15,
                                   max_features=12,
                                   min_samples_leaf=12,
                                   min_samples_split=150,
                                   n_estimators=120,
                                   oob_score=True,
                                   n_jobs=-1)
    
    model.fit(X_train, y_train)
    
    train, test = X_train.copy(), X_test.copy()
    train['Label'] = y_train.values
    test['Label'] = y_test.values
    
    ## predict on training set
    pred = model.predict_proba(X_train)
    fdr3_train.append(FDR3(train, pred))
    
    ## predict on test set
    pred = model.predict_proba(X_test)
    fdr3_test.append(FDR3(test, pred))
    
    ## predict on oot
    pred = model.predict_proba(X_oot)
    fdr3_oot.append(FDR3(oot, pred))

rf_df = pd.DataFrame(data=[[np.mean(fdr3_train), np.mean(fdr3_test), np.mean(fdr3_oot)]],\
                     columns=['FDR3 train', 'FDR3 test', 'FDR3 oot'])

In [19]:
rf_df

,FDR3 train,FDR3 test,FDR3 oot
0,0.8857,0.84502,0.60223


In [24]:
rf_result = pd.DataFrame(dict(zip(['FDR3 train', 'FDR3 test', 'FDR3 oot'],[fdr3_train,fdr3_test,fdr3_oot])))
rf_result 

,FDR3 train,FDR3 test,FDR3 oot
0,0.8883,0.8598,0.6034
1,0.8866,0.8122,0.6034
2,0.8920,0.8525,0.5922
3,0.8829,0.8548,0.5922
4,0.8785,0.8049,0.6089
5,0.9011,0.8601,0.5810
6,0.8930,0.8369,0.6145
7,0.8697,0.8581,0.6089
8,0.8934,0.8467,0.6089
9,0.8715,0.8642,0.6089


In [25]:
# rf_result.to_excel('rf_result.xlsx')

In [24]:
rf_df

,FDR3 train,FDR3 test,FDR3 oot
0,0.8939,0.84137,0.59834
